<a href="https://colab.research.google.com/github/Degananda264/Movie-Recommendation-system/blob/master/Content_Based_Movie_Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install contractions

     |████████████████████████████████| 317kB 3.8MB/s 
     |████████████████████████████████| 245kB 16.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=68212e99c5ea7fdd4cc335650d1cda766ec2a4b26955e24a500618cf496bdff0
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [10]:
import pandas as pd
import numpy as np

In [11]:
df=pd.read_csv("movies.csv")

In [12]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [13]:
df.shape

(4803, 20)

In [14]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [15]:
df = df[['title', 'tagline', 'overview', 'popularity']]
df.tagline.fillna(" ",inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']

In [22]:
df.head(1)

,title,tagline,overview,popularity,description
0,Avatar,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora. In the 22nd centur...


In [21]:
df['description'][0]

'Enter the World of Pandora. In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [23]:
df.dropna(inplace=True)
df = df.sort_values(by=['popularity'], ascending=False)

In [24]:
df.head()

,title,tagline,overview,popularity,description
546,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses M..."
95,Interstellar,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...
788,Deadpool,Witness the beginning of a happy ending,Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending Deadpo...
94,Guardians of the Galaxy,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere. Light years from E...
127,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day. An apocalyptic story set in...


In [28]:
import nltk
import re
import numpy as np
import contractions
nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('english')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**PreProcessing the Documents**

In [29]:
  def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [30]:
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(list(df['description']))
len(norm_corpus)

4800

In [32]:
norm_corpus

array(['gru history bad bosses minions stuart kevin bob recruited scarlet overkill supervillain alongside inventor husband herb hatches plot take world',
       'mankind born earth never meant die interstellar chronicles adventures group explorers make use newly discovered wormhole surpass limitations human space travel conquer vast distances involved interstellar voyage',
       'witness beginning happy ending deadpool tells origin story former special forces operative turned mercenary wade wilson subjected rogue experiment leaves accelerated healing powers adopts alter ego deadpool armed new abilities dark twisted sense humor deadpool hunts man nearly destroyed life',
       ...,
       'one way 100 fools stand way hitchhiker named martel gordone gets fight two bikers prostitute one bikers killed gordone arrested sent prison joins prisons boxing team effort secure early parole establish dominance prisons toughest gang',
       'dare go man affair married woman dropped wrong street go

**Extract TF-IDF Features**

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix=tfidf_matrix.toarray()

In [49]:
tfidf_matrix.shape

(4800, 20472)

In [50]:
columns=tf.get_feature_names()

In [51]:
len(columns)

20472

In [52]:
print(columns)

['00', '00 agent', '007', '007 back', '10', '10 million', '10 years', '100', '100 million', '100 years', '1000', '100000', '1000000', '10th', '10year', '10year old', '10yearold', '10yearold boy', '11', '11 years', '11th', '11yearold', '11yearold girl', '12', '12 year', '12 years', '1200', '12th', '12year', '12yearold', '13', '13 years', '13yearold', '13yearold boy', '14', '14th', '14th century', '14year', '14year old', '14yearold', '15', '15 years', '150', '150 million', '15th', '15year', '15yearold', '16', '16 year', '16th', '16th century', '16yearold', '16yearold boy', '16yearold girl', '17', '17 yearold', '17 years', '170000', '17th', '17th century', '17yearold', '17yearold son', '18', '18 yearold', '180', '1800s', '1850s', '1856', '1863', '1890', '1890s', '18th', '18th birthday', '18th century', '18yearold', '19', '19 years', '1900', '1900s', '1910', '1914', '1919', '1920s', '1925', '1927', '1930s', '1933', '1937', '1939', '1940', '1940s', '1941', '1942', '1944', '1945', '1946', '1

In [54]:
tfidf_matrix[0]

array([0., 0., 0., ..., 0., 0., 0.])

**Document Similarity**

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
doc_sim=cosine_similarity(tfidf_matrix)


In [58]:
doc_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00964634],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.00796282],
       [0.        , 0.        , 1.        , ..., 0.        , 0.02712572,
        0.00934032],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.02712572, ..., 0.        , 1.        ,
        0.01082741],
       [0.00964634, 0.00796282, 0.00934032, ..., 0.        , 0.01082741,
        1.        ]])

In [60]:
doc_sim_df=pd.DataFrame(doc_sim)
doc_sim_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.000000,0.000000,0.000000,0.006070,0.008067,0.000000,0.000000,0.0,0.000000,0.025531,0.008554,0.018111,0.000000,0.000000,0.000000,0.000000,0.007439,0.010454,0.000000,0.000000,0.008190,0.008365,0.010035,0.000000,0.000000,0.050976,0.006502,0.0,0.010728,0.000000,0.006908,0.000000,0.167573,0.000000,0.000000,0.000000,0.000000,0.009191,0.053475,...,0.000000,0.009711,0.006508,0.000000,0.0,0.000000,0.0,0.028409,0.0,0.000000,0.000000,0.008870,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033246,0.000000,0.000000,0.000000,0.000000,0.000000,0.034092,0.018754,0.000000,0.037924,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.009646
1,0.000000,1.000000,0.000000,0.017839,0.007967,0.000000,0.000000,0.012501,0.0,0.014840,0.000000,0.000000,0.000000,0.000000,0.012814,0.000000,0.000000,0.024144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008101,0.000000,0.000000,0.016898,0.0,0.017789,0.000000,0.008885,0.009432,0.000000,0.000000,0.014947,0.000000,0.000000,0.000000,0.022738,...,0.000000,0.000000,0.019783,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.011407,0.011409,0.000000,0.011632,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.015329,0.0,0.008367,0.000000,0.000000,0.000000,0.021596,0.000000,0.000000,0.000000,0.000000,0.017561,0.0,0.019152,0.000000,0.000000,0.000000,0.000000,0.007963
2,0.000000,0.000000,1.000000,0.000000,0.017176,0.000000,0.000000,0.000000,0.0,0.024326,0.005471,0.018038,0.000000,0.000000,0.000000,0.005099,0.004985,0.004705,0.000000,0.004843,0.000000,0.017026,0.000000,0.003672,0.003984,0.030998,0.006959,0.000000,0.0,0.006117,0.000000,0.019548,0.020365,0.009213,0.028467,0.010515,0.004198,0.006311,0.015154,0.012698,...,0.020597,0.004723,0.000000,0.016673,0.0,0.000000,0.0,0.000000,0.0,0.006625,0.000000,0.000000,0.006972,0.000000,0.010574,0.0,0.008222,0.008604,0.012782,0.015353,0.006259,0.000000,0.0,0.000000,0.000000,0.010555,0.000000,0.000000,0.000000,0.000000,0.000000,0.006903,0.005023,0.0,0.012893,0.000000,0.025975,0.000000,0.027126,0.009340
3,0.000000,0.017839,0.000000,1.000000,0.000000,0.022414,0.000000,0.000000,0.0,0.037207,0.000000,0.000000,0.000000,0.000000,0.027958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012232,0.017893,0.043639,0.000000,0.000000,0.000000,0.0,0.023127,0.000000,0.000000,0.000000,0.000000,0.000000,0.009324,0.022995,0.000000,0.000000,0.044476,...,0.000000,0.000000,0.014029,0.030561,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.056761,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.018486,0.000000,0.000000,0.000000,0.060846,0.025035,0.0,0.036237,0.030516,0.022605,0.000000,0.000000,0.000000
4,0.006070,0.007967,0.017176,0.000000,1.000000,0.004672,0.000000,0.064572,0.0,0.000000,0.000000,0.036611,0.023879,0.014421,0.012316,0.019834,0.008170,0.004308,0.025459,0.000000,0.008907,0.023736,0.011341,0.003009,0.000000,0.026928,0.000000,0.010026,0.0,0.019557,0.000000,0.028521,0.016018,0.011311,0.016912,0.022974,0.054142,0.013767,0.027397,0.005416,...,0.005088,0.015097,0.030759,0.013663,0.0,0.012624,0.0,0.000000,0.0,0.000000,0.027018,0.012025,0.005714,0.000000,0.017230,0.0,0.022327,0.007051,0.038014,0.011783,0.018612,0.021468,0.0,0.003767,0.000000,0.008650,0.009499,0.000000,0.012749,0.000000,0.022056,0.019659,0.036850,0.0,0.015824,0.000000,0.076022,0.004515,0.043469,0.011464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [66]:
movies_list=df['title'].values

In [67]:
movies_list,movies_list.shape

(array(['Minions', 'Interstellar', 'Deadpool', ..., 'Penitentiary',
        'Alien Zone', 'America Is Still the Place'], dtype=object), (4800,))

**Find Top Similar Movies for a Sample Movie**

**Find movie ID**

In [70]:
movie_idx = np.where(movies_list == 'Penitentiary')[0][0]
movie_idx

4797

**Get Movie Similarities**

In [71]:
movie_similarities=doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0., 0., 0., ..., 1., 0., 0.])

**From the above Similarities taking TOP 5 Similarities**

In [82]:
similar_movie_idxs=np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([1608, 1037, 2023, 3354, 3911])

In [83]:
similar_movies=movies_list[similar_movie_idxs]
similar_movies

array(['The Texas Chainsaw Massacre: The Beginning', 'Wild Hogs',
       "Child's Play 2", 'Crazy/Beautiful', 'Wicked Blood'], dtype=object)

**Build a movie recommender function to recommend top 5 similar movies for any movie**

In [84]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [85]:

popular_movies = ['Minions', 'Interstellar', 'Deadpool', 'Jurassic World', 'Pirates of the Caribbean: The Curse of the Black Pearl',
              'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Terminator Genisys', 
              'Captain America: Civil War', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice', 
              'Pulp Fiction', 'The Godfather', 'The Shawshank Redemption', 'The Lord of the Rings: The Fellowship of the Ring',  
              'Harry Potter and the Chamber of Secrets', 'Star Wars', 'The Hobbit: The Battle of the Five Armies',
              'Iron Man']

In [86]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()


Movie: Minions
Top 5 recommended Movies: ['Despicable Me 2' 'Despicable Me'
 'Teenage Mutant Ninja Turtles: Out of the Shadows' 'Superman'
 'Rise of the Guardians']

Movie: Interstellar
Top 5 recommended Movies: ['Gattaca' 'Space Pirate Captain Harlock' 'Space Cowboys'
 'Starship Troopers' 'Final Destination 2']

Movie: Deadpool
Top 5 recommended Movies: ['Silent Trigger' 'Underworld: Evolution' 'Bronson' 'Shaft' 'Don Jon']

Movie: Jurassic World
Top 5 recommended Movies: ['Jurassic Park' 'The Lost World: Jurassic Park'
 "National Lampoon's Vacation" 'The Nut Job' 'Vacation']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ["Pirates of the Caribbean: Dead Man's Chest"
 'Pirates of the Caribbean: On Stranger Tides' 'The Pirate'
 'The Pirates! In an Adventure with Scientists!' 'Joyful Noise']

Movie: Dawn of the Planet of the Apes
Top 5 recommended Movies: ['Battle for the Planet of the Apes' 'Groove' 'The Other End of the Line'
 'Chicago Overcoat